In [ ]:
# coding: utf-8

# # Accessing the Datapunt Amsterdam Panorama API
# 
# Below a small example script that grabs a random sample of panoramas from the Datapunt Amsterdam Panorama data set.

# In[1]:


import random
import os
import sys
import requests
import json
from pathlib import Path
from osgeo import ogr

API_URL = 'https://api.data.amsterdam.nl/panorama'
ENDPOINT = '/recente_opnames/2018/'
#ENDPOINT = '/opnamelocatie/'

# Set the following to an appropriate value:
#TARGET_PATH = '/Users/data/Documents/FotosAmsterdam/' # note the slash at the end
TARGET_PATH = Path('/Users/datalab1/Lars/Panorama_Amsterdam/Target/test')
# path to external HD
#TARGET_PATH=Path('D:Amsterdam_Panoramas/')

# In[ ]:
def progressBar(value,endvalue,idx):
    bar_length=25
    percent = float(value)/endvalue
    arrow   = '-'*int(round(percent*bar_length)-1)+'>'
    spaces1 = ' '*int(bar_length-len(arrow))
    spaces2 = ' '*int(len(str(endvalue))-len(str(value)))
    spaces0 = ' '*int(7-len(str(idx)))
    
    sys.stdout.write('\rIdx: {0} [{1}] Currently at panorama {2} out of {3}'.format(spaces0+str(idx),arrow+spaces1,spaces2+str(value),str(endvalue)))
    sys.stdout.flush()
    

def check_how_many(url):
    """
    Access HAL-JSON count from first results page in API.
    """
    results = requests.get(url)
    data = json.loads(results.text)
    return data['count']

def get_panorama_detail(dataset_url, n):
    """
    Grab the n-th panorama in the dataset.
    """
    parameters = {
        'page_size': 1,
        'page': n
    }
    
    r = requests.get(dataset_url, params=parameters)
    data = json.loads(r.text)
    return data['results'][0]

def grab_panoramas(n=300000):
    """
    Randomly sample n panoramas from the panorama dataset.
    """
    dataset_url = API_URL + ENDPOINT
    print('Randomly sampling {} panoramas.'.format(n))
    n_panoramas = check_how_many(dataset_url)
    print('There are {} panoramas int the {} dataset.'.format(n_panoramas, dataset_url))
    assert n <= n_panoramas
    
    # Determine which panoramas to access:
    # random.seed(10) # uncomment for deterministic behavior (handy for debugging)
    indices = list(range(1, n_panoramas + 1)) # not zero based indexing
    random.shuffle(indices)
    to_access = indices[0:n]
    
    # Access the panoramas and download the 2000px equirectangular projected image.
    print('Downloading to {} ...'.format(TARGET_PATH))
    for i, idx in enumerate(to_access):
        #if i % 100 == 0:
        #    print('Currently at panorama {} out of {}'.format(i, n))
        progressBar(i,n,idx)
        try:
            details = get_panorama_detail(dataset_url, idx)
            state= checkifinring(details)
            if state==True:
                coord1 = str(details['geometrie']['coordinates'][0])
                coord2 = str(details['geometrie']['coordinates'][1])
                name="Foto_"+str(idx)+"_"+coord1+"_"+coord2+".jpg"
                print(name)
                if os.path.exists(os.path.join(TARGET_PATH, name)):
                    continue
                image_url = details['image_sets']['equirectangular']['small']

                #print(idx)
                download(TARGET_PATH, image_url,name)
        except Exception as e:
            with open(os.path.join(TARGET_PATH,'Errors.txt'),'a') as f:
                f.write('There was an error while opening and downloading panorama with Id: '+str(idx)+'\n')
                f.write(str(e))
                f.write('\n\n')
            pass
        print(details['geometrie']['coordinates'][:2])
    print('Done downloading.')


def download(TARGET_PATH, image_url,name):
    """
    Download and save a panorama file, in directory structure mimicking url structure.
    """
    assert TARGET_PATH # be sure to set the path to download to, otherwise we crash here!
    if not os.path.exists(TARGET_PATH):
        os.makedirs(TARGET_PATH)
        
    r = requests.get(image_url)
    print(image_url)
    assert r.status_code == 200
    
    target_image_path = os.path.join(TARGET_PATH, name)
    #print('Downloading to: ', target_image_path)
    with open(target_image_path, 'wb') as f_out:
        r.raw.decode_content = True
        for chunk in r.iter_content(1024):
            f_out.write(chunk)

def checkifhighway(details): # Basic function to check if point of picture in in single shapefile
    # Load driver and shapefile
    drv = ogr.GetDriverByName('ESRI Shapefile')
    file=r"C:\Users\datalab1\Lars\Panorama_Amsterdam\shapefiles\shapefile_groot\Groot_polygoon-polygon.shp"
    file=r"C:\Users\datalab1\Lars\Panorama_Amsterdam\shapefiles\shapefile_testbrug\Testbrug-polygon.shp"
    ds_in = drv.Open(file,0)    
    lyr_in = ds_in.GetLayer(0)    #Get the shape file's first layer
    f1 = lyr_in.GetFeature(0)

    # Create point based on coordinates from picture
    lon=details['geometrie']['coordinates'][1]
    lat=details['geometrie']['coordinates'][0]
    pt = ogr.Geometry(ogr.wkbPoint)
    pt.AddPoint(lat, lon)   # watch out: first longitude, then latitude here!
    print(pt)

    # Check if point in shape, if so state == True and save picture
    state= f1.geometry().Contains(pt)
    print('state=',state)
    return state

def checkifinring(details):
    # Load driver and shapefile
    drv = ogr.GetDriverByName('ESRI Shapefile')
    #file=r"C:\Users\datalab1\Lars\Panorama_Amsterdam\shapefiles\shapefile_groot\Groot_polygoon-polygon.shp"
    file_binnen=r"C:/Users/datalab1/Lars/Panorama_Amsterdam/shapefiles/shapefile_ringbinnen/Ring_Binnen-polygon.shp"
    file_buiten=r"C:/Users/datalab1/Lars/Panorama_Amsterdam/shapefiles/shapefile_ringbuiten/Ring_Buiten-polygon.shp"
    ds_in_binnen = drv.Open(file_binnen,0)
    ds_in_buiten = drv.Open(file_buiten,0)
    lyr_in_binnen = ds_in_binnen.GetLayer(0)    #Get the shape file's first layer
    lyr_in_buiten = ds_in_buiten.GetLayer(0)    #Get the shape file's first layer
    f1_binnen = lyr_in_binnen.GetFeature(0)
    f1_buiten = lyr_in_buiten.GetFeature(0)

    # Create point based on coordinates from picture
    lon=details['geometrie']['coordinates'][1]
    lat=details['geometrie']['coordinates'][0]
    pt = ogr.Geometry(ogr.wkbPoint)
    pt.AddPoint(lat, lon)   # watch out: first longitude, then latitude here!

    # Check if point in ring, if so state is True
    in_binnen= f1_binnen.geometry().Contains(pt)
    in_buiten= f1_buiten.geometry().Contains(pt)

    if in_buiten==True and in_binnen==False:
        state=True
    else:
        state=False
    print('state=',state)
    return state

def main():
    grab_panoramas()

main()


Randomly sampling 300000 panoramas.
There are 719192 panoramas int the https://api.data.amsterdam.nl/panorama/recente_opnames/2018/ dataset.
Idx:  238441 [>                        ] Currently at panorama      0 out of 300000state= False
[4.87667674875257, 52.3180860092236]
Idx:  492142 [>                        ] Currently at panorama      1 out of 300000state= False
[4.82247490924831, 52.3807742899685]
Idx:   51154 [>                        ] Currently at panorama      2 out of 300000state= False
[5.2562776963375, 52.380210365728]
Idx:   25837 [>                        ] Currently at panorama      3 out of 300000state= False
[5.24789320220141, 52.3604089503916]
Idx:  194589 [>                        ] Currently at panorama      4 out of 300000state= False
[4.86082819565088, 52.2805708021744]
Idx:  159012 [>                        ] Currently at panorama      5 out of 300000state= False
[5.23432558488751, 52.3820555245603]
Idx:  435680 [>                        ] Currently at panorama 

Idx:  133531 [>                        ] Currently at panorama     60 out of 300000state= False
[4.85886531267448, 52.2800759274924]
Idx:  610846 [>                        ] Currently at panorama     61 out of 300000state= False
[4.91642643823363, 52.3870702700749]
Idx:  480390 [>                        ] Currently at panorama     62 out of 300000state= False
[4.84087639137104, 52.365004265965]
Idx:  360291 [>                        ] Currently at panorama     63 out of 300000state= False
[4.90529361212413, 52.3558859496225]
Idx:  463233 [>                        ] Currently at panorama     64 out of 300000state= False
[4.79243260340328, 52.3816901520826]
Idx:  447977 [>                        ] Currently at panorama     65 out of 300000state= False
[4.79278792611757, 52.3529748364285]
Idx:  375013 [>                        ] Currently at panorama     66 out of 300000state= False
[4.88111428130158, 52.3516875595774]
Idx:  693428 [>                        ] Currently at panorama     67 

Idx:  502041 [>                        ] Currently at panorama    118 out of 300000state= False
[4.82069521593843, 52.3844147039329]
Idx:  622956 [>                        ] Currently at panorama    119 out of 300000state= False
[4.90107992610021, 52.4042672307912]
Idx:  285054 [>                        ] Currently at panorama    120 out of 300000state= False
[4.89033521674822, 52.3587268764998]
Idx:  313714 [>                        ] Currently at panorama    121 out of 300000state= False
[4.98537251699949, 52.3604957209281]
Idx:  479174 [>                        ] Currently at panorama    122 out of 300000state= False
[4.8406006827412, 52.3600422415087]
Idx:  320773 [>                        ] Currently at panorama    123 out of 300000state= False
[4.93627339232808, 52.3583941829813]
Idx:  400201 [>                        ] Currently at panorama    124 out of 300000state= False
[4.86767462870568, 52.3774728690055]
Idx:  303841 [>                        ] Currently at panorama    125 

Idx:  497314 [>                        ] Currently at panorama    180 out of 300000state= False
[4.82842414798291, 52.38037616886]
Idx:  485682 [>                        ] Currently at panorama    181 out of 300000state= False
[4.8277106913582, 52.3726961632802]
Idx:   39921 [>                        ] Currently at panorama    182 out of 300000state= False
[5.19494483169872, 52.3427349797354]
Idx:  301287 [>                        ] Currently at panorama    183 out of 300000state= False
[4.94669800938871, 52.3740159673956]
Idx:  251131 [>                        ] Currently at panorama    184 out of 300000state= False
[4.76983118396065, 52.2583261779049]
Idx:  245276 [>                        ] Currently at panorama    185 out of 300000state= False
[4.79423754996689, 52.2727318471505]
Idx:  554423 [>                        ] Currently at panorama    186 out of 300000state= False
[4.97721286791901, 52.3065378268355]
Idx:  207365 [>                        ] Currently at panorama    187 ou

Idx:  165347 [>                        ] Currently at panorama    240 out of 300000state= False
[5.25243459141272, 52.404520766401]
Idx:  575710 [>                        ] Currently at panorama    241 out of 300000state= False
[4.88270665553403, 52.4110841312259]
Idx:  458871 [>                        ] Currently at panorama    242 out of 300000state= False
[4.75590789967373, 52.3594462705967]
Idx:  667905 [>                        ] Currently at panorama    243 out of 300000state= False
[5.28348519578704, 52.3581205085579]
Idx:  373621 [>                        ] Currently at panorama    244 out of 300000state= False
[4.87701741291654, 52.3490158827137]
Idx:  112244 [>                        ] Currently at panorama    245 out of 300000state= False
[5.26993173050193, 52.3777017633062]
Idx:  482413 [>                        ] Currently at panorama    246 out of 300000state= False
[4.79974734904665, 52.3754009053398]
Idx:   94590 [>                        ] Currently at panorama    247 

...

Idx:  203169 [>                        ] Currently at panorama    896 out of 300000state= False
[4.82092855887312, 52.2784587834958]
Idx:  624826 [>                        ] Currently at panorama    897 out of 300000state= False
[4.89723709242859, 52.4027691780727]
Idx:  689714 [>                        ] Currently at panorama    898 out of 300000state= False
[5.15423121031548, 52.3571591043281]
Idx:  701717 [>                        ] Currently at panorama    899 out of 300000state= False
[5.23531838607291, 52.3382077036572]
Idx:  281546 [>                        ] Currently at panorama    900 out of 300000state= False
[4.90798525207444, 52.3716031040972]
Idx:  219984 [>                        ] Currently at panorama    901 out of 300000state= False
[4.85997431954851, 52.2934633548427]
Idx:  245947 [>                        ] Currently at panorama    902 out of 300000state= False
[4.80603761774339, 52.2810444212319]
Idx:  688593 [>                        ] Currently at panorama    903

Idx:  639185 [>                        ] Currently at panorama    958 out of 300000state= False
[4.88457922975412, 52.4118972246088]
Idx:   50675 [>                        ] Currently at panorama    959 out of 300000state= False
[5.2481672419206, 52.3822734561793]
Idx:  551424 [>                        ] Currently at panorama    960 out of 300000state= False
[4.98573986342808, 52.2904274890834]
Idx:  609443 [>                        ] Currently at panorama    961 out of 300000state= False
[4.92391664169864, 52.3890316196866]
Idx:  479518 [>                        ] Currently at panorama    962 out of 300000state= False
[4.83585877415528, 52.3651113512519]
Idx:  586618 [>                        ] Currently at panorama    963 out of 300000state= False
[4.92516927955535, 52.3978446485018]
Idx:  273578 [>                        ] Currently at panorama    964 out of 300000state= False
[4.89621122341011, 52.3789345739734]
Idx:   78626 [>                        ] Currently at panorama    965 

Idx:  581679 [>                        ] Currently at panorama   1017 out of 300000state= False
[4.95602368422391, 52.3943634000975]
Idx:  423429 [>                        ] Currently at panorama   1018 out of 300000state= False
[4.83868596271553, 52.349442609314]
Idx:  368480 [>                        ] Currently at panorama   1019 out of 300000state= False
[4.90558691428264, 52.3463962826136]
Idx:  694309 [>                        ] Currently at panorama   1020 out of 300000state= False
[5.22855958343614, 52.331411854133]
Idx:  710081 [>                        ] Currently at panorama   1021 out of 300000state= False
[5.2260850021511, 52.3472140346378]
Idx:  285149 [>                        ] Currently at panorama   1022 out of 300000state= False
[4.8920001566251, 52.3626984893666]
Idx:  402443 [>                        ] Currently at panorama   1023 out of 300000state= False
[4.85464244536135, 52.3770757186942]
Idx:  144178 [>                        ] Currently at panorama   1024 out

Idx:  718220 [>                        ] Currently at panorama   1077 out of 300000state= False
[5.19674799781654, 52.351465248789]
Idx:  547112 [>                        ] Currently at panorama   1078 out of 300000state= False
[4.95597628605442, 52.2787184345382]
Idx:  184856 [>                        ] Currently at panorama   1079 out of 300000state= False
[4.86287323306678, 52.3026373784627]
Idx:  114062 [>                        ] Currently at panorama   1080 out of 300000state= False
[5.21130310235178, 52.3670726617695]
Idx:  439113 [>                        ] Currently at panorama   1081 out of 300000state= False
[4.76327451975279, 52.3558291890209]
Idx:   92931 [>                        ] Currently at panorama   1082 out of 300000state= False
[5.19039404416781, 52.3385239988998]
Idx:  309921 [>                        ] Currently at panorama   1083 out of 300000state= False
[4.9928584840465, 52.3590594107169]
Idx:  426128 [>                        ] Currently at panorama   1084 o

Idx:  594903 [>                        ] Currently at panorama   1137 out of 300000state= False
[4.87746242390455, 52.3597328295327]
Idx:  105920 [>                        ] Currently at panorama   1138 out of 300000state= False
[5.16934804807995, 52.3778337461157]
Idx:  718669 [>                        ] Currently at panorama   1139 out of 300000state= False
[5.20010272158718, 52.3551992236167]
Idx:  479454 [>                        ] Currently at panorama   1140 out of 300000state= False
[4.83879456944745, 52.3642426152652]
Idx:  412349 [>                        ] Currently at panorama   1141 out of 300000state= False
[4.85567183329061, 52.3934842760111]
Idx:  411200 [>                        ] Currently at panorama   1142 out of 300000state= False
[4.86867852400872, 52.3834467826077]
Idx:  477784 [>                        ] Currently at panorama   1143 out of 300000state= False
[4.83213042576495, 52.3674823087442]
Idx:  441836 [>                        ] Currently at panorama   1144

In [136]:
from osgeo import ogr

lon=5.139154
lat=52.325277

# lat=52.360715
# lon=4.930261

lat=52.33527778
lon=4.89277778

# Load driver and shapefile
drv = ogr.GetDriverByName('ESRI Shapefile')
#file=r"C:\Users\datalab1\Lars\Panorama_Amsterdam\shapefiles\shapefile_groot\Groot_polygoon-polygon.shp"
file_binnen=r"C:/Users/datalab1/Lars/Panorama_Amsterdam/shapefiles/shapefile_ringbinnen/Ring_Binnen-polygon.shp"
file_buiten=r"C:/Users/datalab1/Lars/Panorama_Amsterdam/shapefiles/shapefile_ringbuiten/Ring_Buiten-polygon.shp"
ds_in_binnen = drv.Open(file_binnen,0)
ds_in_buiten = drv.Open(file_buiten,0)
lyr_in_binnen = ds_in_binnen.GetLayer(0)    #Get the shape file's first layer
lyr_in_buiten = ds_in_buiten.GetLayer(0)    #Get the shape file's first layer
f1_binnen = lyr_in_binnen.GetFeature(0)
f1_buiten = lyr_in_buiten.GetFeature(0)

# Create point based on coordinates from picture
# lon=details['geometrie']['coordinates'][1]
# lat=details['geometrie']['coordinates'][0]
pt = ogr.Geometry(ogr.wkbPoint)
pt.AddPoint(lon, lat)   # watch out: first longitude, then latitude here!

# Check if point in ring, if so state is True
in_binnen= f1_binnen.geometry().Contains(pt)
in_buiten= f1_buiten.geometry().Contains(pt)

if in_buiten==True and in_binnen==False:
    state=True
else:
    state=False
print(state)

True
